# Programming and Data Analysis

> Web Scraping with Python

Kuo, Yao-Jen <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com/)

In [1]:
import requests
import json
import re
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

## About web scraping

## What is web scraping?

> Web scraping is extracting data directly from World Wide Web directly using the Hypertext Transfer Protocol(HTTP). While web scraping can be done manually by a human(via a browser.) The term typically refers to automated processes implemented using a bot or web crawler. It is a form of copying, in which specific data is gathered and copied from the web, typically into a central local database or spreadsheet, for later retrieval or analysis.

Source: <https://en.wikipedia.org/wiki/Web_scraping>

## Core missions involved in web scraping

- Transferring data.
- Parsing data.

## What is transferring data?

Trasferring data is the underlying network protocol between a browser(or a web scraping script) and a server that enables exchange of hypermedia documents on the Web. The technical term for transferring data is the **HyperText Transfer Protocol (HTTP)**.

## 2 ways involved in transferring data

1. To a server: requesting data.
2. From a server: responding data.

## Types of HTTP request methods to a server involved in web scraping

- GET method.
- POST method.

Source: <https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods>

## What is a GET method

> The GET method requests a representation of the specified resource, e.g. reading a blog post.

## What is a POST method

> The POST method is used to submit an entity to the specified resource, often causing a change in state or side effects on the server, e.g. writing a blog post.

## To transfer data between browser and server looks simple and straight-forward

- We enter a Uniform Resource Locator(URL).
- We fill a form then submit.
- We interact with abundant user-interface components.
- ...etc.

## Developer tools

## We can actually see the process of transferring data via the developer tools of a browser

A set of web developer tools built directly into the browser. It can help us edit pages on-the-fly and diagnose problems quickly, which ultimately helps us build better websites, faster.

![Imgur](https://i.imgur.com/3Synk8m.png?1)

## We use Network to see the details of data transfer

1. Open Developer Tools.
2. Click Network.
3. Refresh website.

![Imgur](https://i.imgur.com/OG0Huwj.png?1)

## Each data(file) is a complete process of request and response

- Request
    - Headers
    - Method
- Response
    - Body

## There is a lot of data transferring during web page rendering

- **XHR(XMLHttpRequest)**
- JS
- CSS
- Img
- Media
- Font
- **Doc(HTML documents)**
- WS
- Manifest
- Other

## We can turn off browser's JavaScript to validate where to find the data

- If data disappears, check **XHR**.
- If data still exists, check **Doc**.

## Check which file contains the data we want to scrape

- Preview(Response body rendered in browser)
- Response(body)

## A Chrome browser plug-in to turn JavaScript off

<https://chrome.google.com/webstore/detail/quick-javascript-switcher/geddoclleiomckbhadiaipdggiiccfje>

## Let's see how it works

- <https://ecshweb.pchome.com.tw/search/v3.3>: Data disappears, check **XHR**.
- <https://emap.pcsc.com.tw>: Data disappears, check **XHR**.
- <https://www.imdb.com/chart/top>: Data still exists, check **Doc**.

## Once we've found what we need, check its details

- Headers
    - General
        - Request URL
        - Request Method
    - Request Headers
    - Query String Parameters(if any)
    - Form Data(if any)

![Imgur](https://i.imgur.com/cTva78r.png?1)

![Imgur](https://i.imgur.com/LMVp0m7.png?1)

## But how do we transfer data between Python and server?

We use a third-party library called [Requests](https://requests.readthedocs.io/en/master/).

## Requests

## What is Requests?

> Requests is an elegant and simple HTTP library for Python, built for human being.

Source: <https://requests.readthedocs.io/en/master>

In [2]:
import requests

## If Requests is not installed, we will encounter a `ModuleNotFoundError`

```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'requests'
```

## Use `pip install` at Terminal to install Requests

```bash
pip install requests
```

## Check version and its installation file path

- `__version__` attribute
- `__file__` attribute

In [3]:
print(requests.__version__)
print(requests.__file__)

2.25.1
/Users/woohockhynn/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py


## Requesting data with functions

- `requests.get(request_url)`: Make a request with GET method.
- `requests.post(request_url)`: Make a request with POST method.

## Sending requests with key-word arguments

- `params=query_string_parameters`
- `data=form_data`
- `headers=request_headers`

In [4]:
request_url = "https://ecshweb.pchome.com.tw/search/v3.3/all/results"
query_string_parameters = {
    'q': 'macbook',
    'page': 1,
    'sort': 'rnk/dc'
}
response = requests.get(request_url, params=query_string_parameters)

# print(response.text)
# print(response.json())

In [25]:
request_url = "https://emap.pcsc.com.tw/EMapSDK.aspx"
form_data = {
    'commandid': 'SearchStore',
    'city': '台北市',
    'town': '大安區',
    "roadname": "基隆路三段"
}
response = requests.post(request_url, data=form_data)
print(response.text)  

<?xml version="1.0" encoding="utf-8"?><iMapSDKOutput><MessageID>00000</MessageID><CommandID>SearchStore</CommandID><Status>連線成功</Status><TimeStamp>2022/1/3 下午 04:42:30</TimeStamp><GeoPosition><POIID>972594              </POIID><POIName>長星</POIName><X>121545340</X><Y>25017810</Y><Telno>(02)27395493        </Telno><FaxNo>(02)27397197        </FaxNo><Address>台北市大安區基隆路三段85號</Address><SpecialStore_Kind></SpecialStore_Kind><Store_URL></Store_URL><StoreImageTitle>03ATM,04座位區,05ibon WiFi,12外送咖啡服務,19ibon,21現萃茶,22現蒸地瓜,29烘培坊,32糖果屋,33日本7-ELEVEN限定,34國際精品專櫃,37天素地蔬,39光合帕尼尼,66冷凍交貨便</StoreImageTitle></GeoPosition></iMapSDKOutput>


In [6]:
# 之前是用read_html,只能抓有table標記而已，把他們都換成data frame.
# 如果沒有table 標記，用read_html是沒有什麼用的。


# 這個在檢查DOC的時候有一點特別
request_url = "https://www.imdb.com/chart/top"
request_headers = {
    "accept-language": "en-US,en;q=1.0"
}
response = requests.get(request_url, headers=request_headers)
# print(response.text)

## 第一個Webscrapping的核心任務：Transfering 已經完成

## Week 14: 

## Common attributes and methods to use on `Response` type

- `status_code` attribute to validate HTTP response status codes.
- `text` attribute to extract the response content as a Python `str`.
- `json()` method to parse a JSON format and convert to a Python data structure.

## Next step: parsing data accordingly

- JSON format: call `response.json()` method to generate a Python data structure.
- XML format: use a parser to convert `response.text` to an `Element` class.
- HTML format: use a parser to convert `response.text` to a `BeautifulSoup` class.

## JSON format

## What is JSON?

> JavaScript Object Notation (JSON) is a standard text-based format for representing structured data based on JavaScript object syntax. It is commonly used for transmitting data in web applications (e.g., sending some data from the server to the client's browser, so it can be displayed on a web page, or vice versa).

Source: [mozilla.org](https://developer.mozilla.org/en-US/docs/Learn/JavaScript/Objects/JSON)

## How to validate if a response belongs to JSON format?

- Looking into Preview(Response body rendered in browser).
- Looking into Response(body).

## JSON exists as a string

- Key-Value Storage layout, quite similar to a Python `dict`.
- Array layout, quite similar to a Python `list`.

## Let's see how a JSON format looks like

<https://ecshweb.pchome.com.tw/search/v3.3>: Data disappears after turning JavaScript off, check **XHR**.

In [7]:
request_url = "https://ecshweb.pchome.com.tw/search/v3.3/all/results"
query_str_params = {
    'q': 'macbook',
    'page': 1,
    'sort': 'rnk/dc'
}
response = requests.get(request_url, params=query_str_params)

## Calling the `json` method of `Response` to get the parsed structure

In [8]:
json_format = response.json()
print(type(json_format))
print(json_format.keys())

<class 'dict'>
dict_keys(['QTime', 'totalRows', 'totalPage', 'range', 'cateName', 'q', 'subq', 'token', 'isMust', 'prods'])


## In fact, the `json` method of `Response` is actually calling the `loads` function from standard library `json`

In [9]:
json_format = json.loads(response.text)
print(type(json_format))
print(json_format.keys())

<class 'dict'>
dict_keys(['QTime', 'totalRows', 'totalPage', 'range', 'cateName', 'q', 'subq', 'token', 'isMust', 'prods'])


## JSON could also exist as an array layout

In [10]:
print(type(json_format['prods']))
print(len(json_format['prods']))

for prod in json_format['prods']:
    print(prod['name'])



<class 'list'>
20
MacBook Air 13: Apple M1 chip 8-core CPU and 7-core GPU,256GB-Space Grey (MGN63TA/A)
MacBook Pro 14: M1 Pro chip with 8-core CPU and 14-core GPU, 512GB SSD Space Grey
桌上型鋁合金雙槽立式支架 多用途收納NB筆電架/MacBook支架/書架
【Amachine】TYPE C 3.1極速PD 11合1 HUB(for macbook / Surface / ipad pro)
【MacBook】四合一 支援4K Type C to HDMI USB3.0/PD充電 傳輸擴充轉接集線器
MacBook Air 13: 1.1GHz-10th-Intel Core i3, 256GB - Space Grey (MWTJ2TA/A)
MacBook Air 13: Apple M1 chip 8-core CPU and 7-core GPU,256GB-Gold (MGND3TA/A)
桌上型鋁合金立式支架 多用途收納NB筆電架/MacBook支架/書架/平板架
USB3.0 Type-C 轉 HDMI + VGA +USB 可PD充電 影音傳輸轉換器 Macbook專用多功能四合一集線器
MacBook Pro 14: M1 Pro chip with 8-core CPU and 14-core GPU, 512GB SSD Silver
MacBook Pro 13 :Apple M1 chip 8-core CPU and 8-core GPU,256GB SSD-Silver
USB3.0 Type-C 轉 HDMI 影音傳輸轉換器 Macbook專用多功能四合一集線器
MacBook Pro 16: M1 Max chip with 10‑core CPU and 32‑core GPU, 1TB SSD - Space Grey
MacBook Air 13: Apple M1 chip 8-core CPU and 7-core GPU,256GB-Space Grey (MGN63TA/A)
MacBook Air 13: Apple M1 chip 8

## XML format

## What is XML?

> XML (Extensible Markup Language) is a markup language without predefined tags to use. Instead, we define our own tags for our needs. This is a powerful way to store data in a format that can be stored, searched, and shared.

Source: <https://developer.mozilla.org/en-US/docs/Web/XML>

## How to validate if a response belongs to XML format?

Looking into Response(body) and check if there is a XML format being declared.

## XML exists as a string

Self-defined tags with a hierarchical tree layout.

## Let's see how a XML format looks like

<https://emap.pcsc.com.tw/EMapSDK.aspx>: Data disappears after turning JavaScript off, check **XHR**.

In [11]:
request_url = "https://emap.pcsc.com.tw/EMapSDK.aspx"
form_data = {
    'commandid': 'SearchStore',
    'city': '台北市',
    'town': '大安區'
}
response = requests.post(request_url, data=form_data)
# response.text
len(response.text)

47198

## Use the `fromstring` function from `ET` to get the parsed structure

In [12]:
root = ET.fromstring(response.text)
print(type(root))

<class 'xml.etree.ElementTree.Element'>


## Use XPath to extract data from specific tags

> XPath stands for XML Path Language. It provides a flexible way of addressing (pointing to) different parts of an XML document.

Source: <https://developer.mozilla.org/en-US/docs/Web/XPath>

## How to specify a XPath

<https://www.w3schools.com/xml/xpath_syntax.asp>

In [13]:
# The XPath for POIName
poinames = [e.text for e in root.findall('.//POIName')]
print(poinames)
print(type(poinames))
print(len(poinames))

['大台', '大信', '大敦', '中廣', '仁安', '仕吉', '台科一', '永康', '禾光', '立仁', '光忠', '吉孝', '吉忠', '合旺', '合維', '安居', '安松', '佑安', '技安', '辛亥', '卓聯', '和平東', '和安', '和金', '和泰', '和樂', '延吉', '昇隆', '東門', '欣安和', '欣隆昌', '花市', '金信', '金華', '長星', '阿波羅', '信中', '信安', '信義', '信興', '建安', '建忠', '建南', '建綸', '恆安', '科技站', '科建', '科興', '師大', '泰利', '國館', '崇光', '康福', '教育大學', '統合', '統家', '統領', '通化', '頂好', '頂安', '頂東', '喜悅', '富陽', '復忠', '復昌', '復維', '敦仁', '敦禾', '敦安', '敦信', '敦南', '敦頂', '敦隆', '敦維', '敦親', '森美', '華電', '愛國', '新北科', '新東帝', '新泰順', '新國聯', '溫州', '溫東', '瑞升', '瑞安', '義村', '誠安', '福亭', '鳳翔', '樂安', '樂利', '樂和', '樂隆', '黎元', '豫銘', '錢忠', '靜安', '龍和', '龍延', '龍門', '龍泉', '龍淵', '龍普', '濟南', '臨江', '臨通', '豐安', '懷生', '羅鑫', '麟光', '鑫忠孝', '鑫泰', '鑫通', '鑫富民', '鑫復']
<class 'list'>
116


In [14]:
# The XPath for Address
addresses = [e.text for e in root.findall('.//Address')]
print(addresses)

['台北市大安區羅斯福路三段333巷18號1樓20號1樓(部分)', '台北市大安區信義路三段33號', '台北市大安區敦化南路二段63巷7號1樓', '台北市大安區仁愛路三段25-1號27號', '台北市大安區仁愛路四段27巷1號', '台北市大安區忠孝東路四段223巷42號', '台北市大安區基隆路四段43號1樓', '台北市大安區永康街43號', '台北市大安區和平東路二段63號1樓', '台北市大安區安和路二段74巷1號', '台北市大安區復興南路一段107巷5弄1號1樓', '台北市大安區忠孝東路四段299號', '台北市大安區延吉街72號', '台北市大安區復興南路二段151巷41號', '台北市大安區四維路170巷8號1樓', '台北市大安區安居街33號', '台北市大安區安東街50之2號50之3號50之4號', '台北市大安區忠孝東路三段217巷1弄2號', '台北市大安區和平東路三段97號97之1號1樓', '台北市大安區辛亥路二段57號', '台北市大安區羅斯福路四段1號1樓卓聯大樓', '台北市大安區和平東路一段129之1號', '台北市大安區和平東路三段230號', '台北市大安區和平東路一段91號', '台北市大安區和平東路一段169號', '台北市大安區和平東路三段228巷45號1樓', '台北市大安區延吉街237號', '台北市大安區敦化南路二段238號', '台北市大安區信義路二段198巷6號1樓', '台北市大安區安和路一段47號', '台北市大安區基隆路二段142之1號及142之2號', '台北市大安區建國南路一段274號', '台北市大安區金山南路二段18號1樓', '台北市大安區金華街140號1樓', '台北市大安區基隆路三段85號', '台北市大安區忠孝東路四段222號224號1樓', '台北市大安區信義路三段101號', '台北市大安區大安路一段218號', '台北市大安區信義路四段265巷12弄1號', '台北市大安區信義路四段32號', '台北市大安區敦化南路一段187巷29號', '台北市大安區忠孝東路三段249號', '台北市大安區建國南路二段151巷6之8號', '台北市大安區仁愛路四段151巷33號忠孝東路四段216巷32弄19號21號', '台北市大安區永康街2巷12號1樓', '台北市大安區復興南路二段20

In [15]:
# The XPath for StoreImageTitles
store_image_titles = [e.text for e in root.findall('.//StoreImageTitle')]
print(store_image_titles)

['02廁所,03ATM,04座位區,05ibon WiFi,09霜淇淋,12外送咖啡服務,19ibon,22現蒸地瓜,31博客來,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,37天素地蔬,38嚴選素材冷凍鮮物,66冷凍交貨便', '02廁所,03ATM,04座位區,05ibon WiFi,13台塑有機蔬菜,15Mister Donut甜甜圈,16美妝,19ibon,22現蒸地瓜,24統一生機,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '03ATM,04座位區,05ibon WiFi,12外送咖啡服務,19ibon,22現蒸地瓜,24統一生機,31博客來,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '03ATM,04座位區,05ibon WiFi,13台塑有機蔬菜,19ibon,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '02廁所,03ATM,04座位區,05ibon WiFi,07OPEN! STORE,11千禧血壓站,12外送咖啡服務,19ibon,22現蒸地瓜,24統一生機,25OPEN!兒童閱覽室,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '03ATM,04座位區,05ibon WiFi,12外送咖啡服務,19ibon,22現蒸地瓜,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '03ATM,04座位區,09霜淇淋,12外送咖啡服務,15Mister Donut甜甜圈,19ibon,22現蒸地瓜,33日本7-ELEVEN限定,34國際精品專櫃,37天素地蔬,66冷凍交貨便', '03ATM,05ibon WiFi,12外送咖啡服務,19ibon,33日本7-ELEVEN限定,36精品咖啡', '03ATM,05ibon WiFi,19ibon,22現蒸地瓜,33日本7-ELEVEN限定,34國際精品專櫃,36精品咖啡,66冷凍交貨便', '03ATM,05ibon WiFi,12外送咖啡服務,19ibon,33日本7-ELEVEN限定,66冷凍交貨便', '03ATM,04座位區,22現蒸地瓜,33日本7-ELEVEN限定,34國際精品專櫃,66冷凍

In [16]:
for poiname, address, store_image_title in zip(poinames, addresses, store_image_titles):
    if "霜淇淋" in store_image_title:
        print(f"{poiname}:{address}")

大台:台北市大安區羅斯福路三段333巷18號1樓20號1樓(部分)
台科一:台北市大安區基隆路四段43號1樓
敦信:台北市大安區仁愛路四段122巷50號1樓


## HTML format

## What is HTML?

> HTML (HyperText Markup Language) is a descriptive language that specifies webpage structure. An HTML document is a plaintext document structured with elements. Elements are surrounded by matching opening and closing tags. Each tag begins and ends with angle brackets (<>).

Source: <https://developer.mozilla.org/en-US/docs/Glossary/HTML>

## How to validate if a response belongs to HTML format?

Looking into Response(body) and check if there is a HTML format being declared.

## HTML exists as a string

Predefined tags with a hierarchical tree layout.

## Let's see how a HTML format looks like

<https://www.imdb.com/chart/top>: Data still exists after turning JavaScript off, check **Doc**.

In [17]:
request_url = "https://www.imdb.com/chart/top"
response = requests.get(request_url, headers={"accept-language": "en-US,en;q=1.0"})
# print(response.text)
print(len(response.text))

544723


## Use the `BeautifulSoup` function from `bs4` to get the parsed structure

In [18]:
soup = BeautifulSoup(response.text)
print(type(soup))

<class 'bs4.BeautifulSoup'>


## Use CSS Selector to extract data from specific tags

> A CSS selector is the part of a CSS rule that describes what elements in a document the rule will match. The matching elements will have the rule's specified style applied to them.

Source: <https://developer.mozilla.org/en-US/docs/Glossary/CSS_Selector>

## A CSS selector can be mixed and matched with

1. Tag names.
2. Class attribute in tags.
3. Id attribute in tags.

## How to specify a CSS selector

<https://www.w3schools.com/cssref/css_selectors.asp>

## It is not easy to find CSS selector unless we are a seasoned front-end engineer

A Chrome browser plug-in to help us find the specific CSS selector of element(s): <https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb>

In [24]:
soup.select('.titleColumn a')
len(soup.select('.titleColumn a'))

titles = [e.text for e in soup.select('.titleColumn a')] # The CSS Selector for titles
print(titles)

['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II', 'The Dark Knight', '12 Angry Men', "Schindler's List", 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Good, the Bad and the Ugly', 'The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'Forrest Gump', 'Inception', 'The Lord of the Rings: The Two Towers', 'Star Wars: Episode V - The Empire Strikes Back', 'Spider-Man: No Way Home', 'The Matrix', 'Goodfellas', "One Flew Over the Cuckoo's Nest", 'Seven Samurai', 'Se7en', 'The Silence of the Lambs', "It's a Wonderful Life", 'City of God', 'Life Is Beautiful', 'Saving Private Ryan', 'Star Wars: Episode IV - A New Hope', 'Interstellar', 'Spirited Away', 'The Green Mile', 'Parasite', 'Léon: The Professional', 'Hara-Kiri', 'The Pianist', 'Terminator 2: Judgment Day', 'Back to the Future', 'The Usual Suspects', 'Psycho', 'The Lion King', 'Modern Times', 'Grave of the Fireflies', 'American History X', 'Whiplash', 'Gladiator', 'City Lights',

In [20]:
# soup.select('strong')
# len(soup.select('strong'))

# number_of_user_rating = [int(e.get("title").split()[3].replace(',','')) for e in soup.select('strong')]  
# number_of_user_rating

# ratings = [float(e.text) for e in soup.select('strong')] # The CSS Selector for ratings
# print(ratings)

In [21]:
release_years = [int(re.sub("\(|\)", "", e.text)) for e in soup.select('span.secondaryInfo')] # The CSS Selector for release years
print(release_years)

[1994, 1972, 1974, 2008, 1957, 1993, 2003, 1994, 1966, 2001, 1999, 1994, 2010, 2002, 1980, 2021, 1999, 1990, 1975, 1954, 1995, 1991, 1946, 2002, 1997, 1998, 1977, 2014, 2001, 1999, 2019, 1994, 1962, 2002, 1991, 1985, 1995, 1960, 1994, 1936, 1988, 1998, 2014, 2000, 1931, 2006, 2011, 2006, 1942, 1968, 1954, 1988, 1979, 1979, 2000, 1981, 1940, 2012, 2006, 1957, 1950, 2008, 2018, 1957, 1980, 2018, 1964, 1997, 2003, 2019, 2016, 2017, 2012, 1986, 1984, 2019, 2018, 1981, 1963, 2009, 1995, 1984, 1999, 1995, 2009, 2020, 1997, 1983, 1968, 1985, 1992, 2007, 1958, 1931, 1941, 2012, 2000, 1952, 1959, 2004, 1952, 1948, 1962, 1921, 1987, 2016, 1960, 2010, 2020, 1944, 1927, 1976, 1971, 2011, 1973, 1983, 2000, 2001, 2019, 1955, 1962, 2010, 1965, 2009, 1989, 1995, 1997, 1961, 1985, 1988, 2018, 1950, 2004, 1975, 1950, 1959, 2005, 1992, 1997, 2021, 2004, 2013, 1961, 2007, 1963, 1995, 1948, 2006, 2001, 2009, 1980, 1974, 1988, 1975, 2010, 1998, 2007, 1954, 1925, 2019, 1982, 2017, 1957, 1980, 1999, 1998, 199

In [22]:
len(soup.select('img'))

# alt_title = [e.get('alt') for e in soup.select('img')]   #alt代表如果圖片讀不出來，就會顯示電影名字
# alt_title 

# posters = [e.get("src") for e in soup.select('.posterColumn img')] # The CSS Selector for posters
# print(posters)

250